In [248]:
import qulacs
from functools import reduce
from sympy import Matrix, KroneckerProduct, I, pi, exp, sqrt
import itertools
import sympy as sp
import numpy as np
from typing import List, Dict, Tuple

In [256]:
bit_map = {0: Matrix((1, 0)), 1: Matrix((0, 1))}

X = Matrix([[0, 1], [1, 0]])
Y = Matrix([[0, -I], [I, 0]])
Z = Matrix([[1, 0], [0, -1]])

H = (X + Z)/sqrt(2)
T = Matrix([[1, 0], [0, exp(I * pi/4)]])
S = Matrix([[1, 0], [0, I]])
Id2 = sp.eye(2)

In [79]:
class BasisKet:

    def __init__(self, *label) -> None:
        self.dim = len(label)
        self.label = label
        self.tensor_rep = self.get_mat_rep(doit=False)
        self.mat_rep = self.get_mat_rep(doit=True)
        
    
    def get_mat_rep(self, doit = True) -> Matrix:
        ket_mat = [bit_map[bit] for bit in self.label]
        if doit:
            return reduce(KroneckerProduct, ket_mat).doit()
        else:
            return reduce(KroneckerProduct, ket_mat)

    
    @property
    def T(self):
        return self.mat_rep.T

    def __eq__(self, other):
        if isinstance(other, BasisKet):
            return self.label == other.label

    def __hash__(self) -> int:
        return hash(self.label)

    def __repr__(self) -> str:
        return self.mat_rep.__repr__()
    
    def __str__(self) -> str:
        return self.mat_rep.__str__()

In [102]:
def generate_basis(dim: int) -> List[BasisKet]:

    basis_kets = [[0, 1]] * dim
    basis_kets = map(lambda lab: BasisKet(*lab), itertools.product(*basis_kets))
    basis_kets = list(basis_kets)

    return basis_kets

In [222]:
class ControlledOperator_v1:

    def __init__(self, n: int, controlled_bit_label: tuple, target_bit_pos: int, operatorU: Matrix, condition_on = 1) -> None:

        """
        n: total number of channels
        controlled_bit_label: label of bits that are used as controlled bits. Label of bit starts from 1
        condition_on: activate the gate  
        """
        
        self.n = n
        self.tar_pos = target_bit_pos
        self.operatorU = operatorU
        self.condition_on = condition_on
        self.mat_rep = self.get_mat_rep()

    def get_mat_rep(self):
 
        # C(U)|x1, ..., x_i, ..., x_n > = |x1, ..., U(x_i), ..., x_n > if x1 = ... = x{i-1} = x{i+1} = ... = x_n = 1
        basis_kets = generate_basis(self.n)
        
        def eval_mat_el(i, j):
            
            in_state = basis_kets[j]
            in_state_label = in_state.label

            opp = int(not self.condition_on)

            if opp in in_state_label[:self.tar_pos - 1] + in_state_label[self.tar_pos:]:
                out_state = in_state.mat_rep
            else:
                operator = [sp.eye(2) for _ in range(self.tar_pos-1)] + [self.operatorU] + [sp.eye(2) for _ in range(self.n - self.tar_pos)]
                operator = reduce(KroneckerProduct, operator).doit()
                out_state = operator @ in_state.mat_rep

            
            out_proj = basis_kets[i]
            res = out_proj.mat_rep.T @ out_state
            
            return res

        matrix = [ [eval_mat_el(i, j) for i in range(2**self.n)] for j in range(2**self.n) ]
        
        return Matrix(matrix)

In [279]:
class ControlledOperator:

    def __init__(self, n: int, target_bits_dict: Dict[int, Matrix], controlled_bits_dict: Dict[int, int]) -> None:

        """
        channel label starts from 0
        n: total number of channels
        target_bits_dict: {channel label: gate operator}. If it is free, gate operator = Id2.
        controlled_bits_dict: {channel label: activate_on 0 or 1}
        """
        
        self.n = n
        self.basis_kets = generate_basis(self.n)

        self.target_bits_dict = target_bits_dict
        self.target_bits_idx = self.target_bits_dict.keys()
        
        self.controlled_bits_dict = controlled_bits_dict
        self.controlled_bits_idx = self.controlled_bits_dict.keys()
        self.controlled_bits_val = self.controlled_bits_dict.values()

        self.matrix_rep = self.compute_matrix_rep()
        

    def compute_matrix_rep(self) -> Matrix:

        def matrix_el(i, j):
            
            out_proj = self.basis_kets[i].mat_rep.T
            in_state = self.basis_kets[j]

            in_state_control_bit = tuple([in_state.label[i] for i in self.controlled_bits_idx])

            if in_state_control_bit != tuple(self.controlled_bits_val):
                out_state = in_state.mat_rep
            else:
                operator = [ self.target_bits_dict[i] if i in self.target_bits_dict else Id2 for i in range(self.n)]
                operator = reduce(KroneckerProduct, operator).doit()
                out_state = operator @ in_state.mat_rep
            
            return out_proj @ out_state
            

        matrix = Matrix([[matrix_el(i, j) for j in range(2**self.n)] for i in range(2**self.n)])

        return matrix
        

In [271]:
# CNOT = ControlledOperator_v1(controlled_bit_label = (1, 1), target_bit_pos = 2, operatorU = X, condition_on=0)
# Toffoli = ControlledOperator_v1(controlled_bit_label = (1, 1, 1), target_bit_pos = 1, operatorU = X, condition_on=1)
# CZ = ControlledOperator_v1(controlled_bit_label = (1, 1), target_bit_pos = 2, operatorU = Z, condition_on=1)

In [304]:
CNOT = ControlledOperator(n = 2, target_bits_dict = {1: X}, controlled_bits_dict = {0: 1})

Toffoli_01 = ControlledOperator(n = 3, target_bits_dict = {2: X}, controlled_bits_dict = {0: 1, 1: 1})
Toffoli_02 = ControlledOperator(n = 3, target_bits_dict = {1: X}, controlled_bits_dict = {0: 1, 2: 1})
Toffoli_12 = ControlledOperator(n = 3, target_bits_dict = {0: X}, controlled_bits_dict = {1: 1, 2: 1})

CNOT_c0_t1 = ControlledOperator(n = 3, target_bits_dict = {1: X}, controlled_bits_dict = {0: 1})
CNOT_c0_t2 = ControlledOperator(n = 3, target_bits_dict = {2: X}, controlled_bits_dict = {0: 1})
CNOT_c1_t0 = ControlledOperator(n = 3, target_bits_dict = {0: X}, controlled_bits_dict = {1: 1})
CNOT_c1_t2 = ControlledOperator(n = 3, target_bits_dict = {2: X}, controlled_bits_dict = {1: 1})
CNOT_c2_t0 = ControlledOperator(n = 3, target_bits_dict = {0: X}, controlled_bits_dict = {2: 1})
CNOT_c2_t1 = ControlledOperator(n = 3, target_bits_dict = {1: X}, controlled_bits_dict = {2: 1})

In [335]:
CNOT_c0_t2.matrix_rep @ CNOT_c0_t1.matrix_rep

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0]])

In [317]:
[
    (CNOT_c0_t2.matrix_rep @ BasisKet(0, 0, 0).mat_rep - BasisKet(0, 0, 0).mat_rep == Matrix(tuple([0] * 8))),
    (CNOT_c0_t2.matrix_rep @ BasisKet(0, 0, 1).mat_rep - BasisKet(0, 0, 1).mat_rep == Matrix(tuple([0] * 8))),
    (CNOT_c0_t2.matrix_rep @ BasisKet(0, 1, 0).mat_rep - BasisKet(0, 1, 0).mat_rep == Matrix(tuple([0] * 8))),
    (CNOT_c0_t2.matrix_rep @ BasisKet(0, 1, 1).mat_rep - BasisKet(0, 1, 1).mat_rep == Matrix(tuple([0] * 8))),
    (CNOT_c0_t2.matrix_rep @ BasisKet(1, 0, 0).mat_rep - BasisKet(1, 0, 1).mat_rep == Matrix(tuple([0] * 8))),
    (CNOT_c0_t2.matrix_rep @ BasisKet(1, 0, 1).mat_rep - BasisKet(1, 0, 0).mat_rep == Matrix(tuple([0] * 8))),
    (CNOT_c0_t2.matrix_rep @ BasisKet(1, 1, 0).mat_rep - BasisKet(1, 1, 1).mat_rep == Matrix(tuple([0] * 8))),
    (CNOT_c0_t2.matrix_rep @ BasisKet(1, 1, 1).mat_rep - BasisKet(1, 1, 0).mat_rep == Matrix(tuple([0] * 8))),
]

[True, True, True, True, True, True, True, True]

In [318]:
[
    Toffoli_12.matrix_rep @ BasisKet(0, 0, 0).mat_rep - BasisKet(0, 0, 0).mat_rep == Matrix(tuple([0] * 8)),
    Toffoli_12.matrix_rep @ BasisKet(0, 0, 1).mat_rep - BasisKet(0, 0, 1).mat_rep == Matrix(tuple([0] * 8)),
    Toffoli_12.matrix_rep @ BasisKet(0, 1, 0).mat_rep - BasisKet(0, 1, 0).mat_rep == Matrix(tuple([0] * 8)),
    Toffoli_12.matrix_rep @ BasisKet(0, 1, 1).mat_rep - BasisKet(1, 1, 1).mat_rep == Matrix(tuple([0] * 8)),
    Toffoli_12.matrix_rep @ BasisKet(1, 0, 0).mat_rep - BasisKet(1, 0, 0).mat_rep == Matrix(tuple([0] * 8)),
    Toffoli_12.matrix_rep @ BasisKet(1, 0, 1).mat_rep - BasisKet(1, 0, 1).mat_rep == Matrix(tuple([0] * 8)),
    Toffoli_12.matrix_rep @ BasisKet(1, 1, 0).mat_rep - BasisKet(1, 1, 0).mat_rep == Matrix(tuple([0] * 8)),
    Toffoli_12.matrix_rep @ BasisKet(1, 1, 1).mat_rep - BasisKet(0, 1, 1).mat_rep == Matrix(tuple([0] * 8)),
]

[True, True, True, True, True, True, True, True]

In [313]:
Toffoli_decomp = [
                    KroneckerProduct(Id2, Id2, H),
                    CNOT_c1_t2.matrix_rep,
                    KroneckerProduct(Id2, Id2, T.adjoint()),
                    CNOT_c0_t2.matrix_rep,
                    KroneckerProduct(Id2, Id2, T),
                    CNOT_c1_t2.matrix_rep,
                    KroneckerProduct(Id2, Id2, T.adjoint()),
                    CNOT_c0_t2.matrix_rep,
                    KroneckerProduct(Id2, T.adjoint(), T),
                    CNOT_c0_t1.matrix_rep,
                    KroneckerProduct(Id2, Id2, H),
                    KroneckerProduct(Id2, T.adjoint(), Id2),
                    CNOT_c0_t1.matrix_rep,
                    KroneckerProduct(T, S, Id2)
                ]
Toffoli_decomp = reversed(Toffoli_decomp)
Toffoli_decomp = reduce(lambda g1, g2: g1 @ g2, Toffoli_decomp)

In [314]:
Toffoli_decomp

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 1, 0]])

In [332]:
class QuantumGate:

    def __init__(self, n: int, free_bits_dict: Dict[int, Matrix], target_bits_dict: Dict[int, Matrix], controlled_bits_dict: Dict[int, int]) -> None:

        """
        channel label starts from 0

        n: total number of channels

        free_bits_dict: {channel label: gate operator}

        target_bits_dict: {channel label: gate operator}
        
        controlled_bits_dict: {channel label: activate_on 0 or 1}
        """
        
        self.n = n

        for i in range(self.n):
            if (i not in free_bits_dict) and (i not in target_bits_dict) and (i not in controlled_bits_dict):
                free_bits_dict[i] = Id2

        self.basis_kets = generate_basis(self.n)

        self.free_bits_dict = free_bits_dict
        self.free_bits_idx = free_bits_dict.keys()
        self.free_bits_val = free_bits_dict.values()

        self.target_bits_dict = target_bits_dict
        self.target_bits_idx = self.target_bits_dict.keys()
        
        self.controlled_bits_dict = controlled_bits_dict
        self.controlled_bits_idx = self.controlled_bits_dict.keys()
        self.controlled_bits_val = self.controlled_bits_dict.values()

        self.matrix_rep = self.compute_matrix_rep()
        

    def compute_matrix_rep(self) -> Matrix:

        def matrix_el(i, j):
            
            out_proj = self.basis_kets[i].mat_rep.T
            in_state = self.basis_kets[j]

            in_state_control_bit = tuple([in_state.label[i] for i in self.controlled_bits_idx])

            if in_state_control_bit != tuple(self.controlled_bits_val):
                operator = [self.free_bits_dict[i] if i in self.free_bits_dict else Id2 for i in range(self.n)]
            else:
                activated_gate = self.free_bits_dict|self.target_bits_dict
                operator = [activated_gate[i] if i in activated_gate else Id2 for i in range(self.n)]
            
            operator = reduce(KroneckerProduct, operator).doit()
            out_state = operator @ in_state.mat_rep
            
            return out_proj @ out_state
            

        matrix = Matrix([[matrix_el(i, j) for j in range(2**self.n)] for i in range(2**self.n)])

        return matrix
    

    @classmethod
    def fromTruthTable(cls, truth_table: Dict[Matrix, Matrix]):
        pass
        

In [330]:
tmp = QuantumGate(3, free_bits_dict={}, target_bits_dict={2: X}, controlled_bits_dict={0: 1, 1: 1})

In [ ]:
QuantumGate.fromTruthTable()

In [333]:
tmp.matrix_rep @ tmp.matrix_rep

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 1]])

In [329]:
CNOT_c0_t2.matrix_rep

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 1, 0]])

In [336]:
class QuantumCircuit:

    def __init__(self, gates: List[QuantumGate], reversed_order = True) -> None:

        self.gates = reversed(gates) if reversed_order else gates
        self.matrix_rep = reduce(KroneckerProduct, self.gates).doit()
        
        

In [337]:
CNOT = QuantumGate(n = 2, free_bits_dict={}, target_bits_dict={1: X}, controlled_bits_dict={0: 1})

In [350]:
[
    CNOT.matrix_rep @ KroneckerProduct(X, Id2) @ CNOT.matrix_rep == KroneckerProduct(X, Id2) @ KroneckerProduct(Id2, X),
    CNOT.matrix_rep @ KroneckerProduct(Y, Id2) @ CNOT.matrix_rep == KroneckerProduct(Y, Id2) @ KroneckerProduct(Id2, X),
    CNOT.matrix_rep @ KroneckerProduct(Z, Id2) @ CNOT.matrix_rep == KroneckerProduct(Z, Id2).doit(),
    CNOT.matrix_rep @ KroneckerProduct(Id2, X) @ CNOT.matrix_rep == KroneckerProduct(Id2, X).doit(),
    CNOT.matrix_rep @ KroneckerProduct(Id2, Y) @ CNOT.matrix_rep == KroneckerProduct(Z, Id2) @ KroneckerProduct(Id2, Y),
    CNOT.matrix_rep @ KroneckerProduct(Id2, Z) @ CNOT.matrix_rep == KroneckerProduct(Z, Id2) @ KroneckerProduct(Id2, Z),
]

[True, True, True, True, True, True]

In [352]:
Toffoli_c0c1_t2 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={2: X}, controlled_bits_dict={0: 1, 1: 1})
Toffoli_c0c2_t1 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={1: X}, controlled_bits_dict={0: 1, 2: 1})
Toffoli_c1c2_t0 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={0: X}, controlled_bits_dict={1: 1, 2: 1})

In [ ]:
map(KroneckerProduct, itertools.product([[Toffoli_c0c1_t2.matrix_rep, Toffoli_c0c2_t1.matrix_rep, Toffoli_c1c2_t0.matrix_rep]]*3)).__next__()

In [396]:
Fredkin = Matrix([[1, 0, 0, 0, 0, 0, 0, 0], 
                 [0, 1, 0, 0, 0, 0, 0, 0], 
                 [0, 0, 1, 0, 0, 0, 0, 0],
                 [0, 0, 0, 1, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 0, 1, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1]])
all_combo = list(itertools.product(*[[Toffoli_c0c1_t2.matrix_rep, Toffoli_c0c2_t1.matrix_rep, Toffoli_c1c2_t0.matrix_rep]]*3))
eval_combo = list(map(lambda comb: reduce(lambda m1, m2: m1 @ m2, comb) == Fredkin, all_combo))

In [394]:
[
    Toffoli_c0c1_t2.matrix_rep @ Toffoli_c0c2_t1.matrix_rep @ Toffoli_c0c1_t2.matrix_rep == Fredkin,
    Toffoli_c0c2_t1.matrix_rep @ Toffoli_c0c1_t2.matrix_rep @ Toffoli_c0c2_t1.matrix_rep == Fredkin
]

[True, True]

In [19]:
from openfermion import FermionOperator, MolecularData

In [22]:
FermionOperator(((4, 1), (1, 0)))

1.0 [4^ 1]

In [27]:
(1j)**2

(-1+0j)

In [28]:
from openfermion.hamiltonians import MolecularData

ImportError: cannot import name 'MolecularData' from 'openfermion.hamiltonians' (/usr/local/lib/python3.9/site-packages/openfermion/hamiltonians/__init__.py)

In [31]:
from openfermion import MolecularData

In [34]:
geomertry = [['H', [0, 0, 0]], ['H', [0, 0 , 0.74]]]
basis = 'sto-3g'
multuplicity = 1
charge = 0
h2_mol = MolecularData(geometry=geomertry, basis = basis, multiplicity=multuplicity, charge=charge)

In [47]:
h2_mol.__dict__

{'geometry': [['H', [0, 0, 0]], ['H', [0, 0, 0.74]]],
 'basis': 'sto-3g',
 'multiplicity': 1,
 'charge': 0,
 'description': '',
 'name': 'H2_sto-3g_singlet',
 'filename': '/usr/local/lib/python3.9/site-packages/openfermion/testing/data/H2_sto-3g_singlet',
 'n_atoms': 2,
 'atoms': ['H', 'H'],
 'protons': [1, 1],
 'n_electrons': 2,
 'n_orbitals': None,
 'n_qubits': None,
 'nuclear_repulsion': None,
 'hf_energy': None,
 'orbital_energies': None,
 'mp2_energy': None,
 'cisd_energy': None,
 'fci_energy': None,
 'ccsd_energy': None,
 'general_calculations': {},
 '_canonical_orbitals': None,
 '_overlap_integrals': None,
 '_one_body_integrals': None,
 '_two_body_integrals': None,
 '_cisd_one_rdm': None,
 '_cisd_two_rdm': None,
 '_fci_one_rdm': None,
 '_fci_two_rdm': None,
 '_ccsd_single_amps': None,
 '_ccsd_double_amps': None}